In [ ]:
# ==========================================
# 一键运行脚本 (All-in-One) - 消融实验版本
# ==========================================
import os

# 1. 环境清理与准备
print(">>> [1/5] 清理环境...")
if os.path.exists('/kaggle/working'):
    os.chdir('/kaggle/working')
    !rm -rf ai_project

# 2. 克隆代码仓库
print(">>> [2/5] 克隆仓库...")
# 【重要】如果仓库是 Private，请在下方填入 Token URL
# repo_url = "https://<你的Token>@github.com/yangzeha/ai_project.git"
repo_url = "https://github.com/yangzeha/ai_project.git" 

!git clone $repo_url

if not os.path.exists("ai_project"):
    print("❌ 克隆失败！请检查：1. Internet是否开启 2. 是否需要Token")
else:
    print("✅ 克隆成功")
    %cd ai_project

    # 3. 安装依赖
    print(">>> [3/5] 安装依赖...")
    !pip install -r TSB_CL_Project/requirements.txt
    # 安装 sparsehash (解决 fatal error: sparsehash/dense_hash_map: No such file or directory)
    !apt-get update && apt-get install -y libsparsehash-dev

    # 4. 编译 C++
    print(">>> [4/5] 编译 C++ 核心...")
    %cd Similar-Biclique-Idx-main
    !g++ -O3 main.cpp -o msbe
    !chmod +x msbe
    %cd ..

    # 5. 运行训练 (消融实验: Baseline vs BicliqueGCN, Epochs=50, Tau=2)
    print(">>> [5/5] 开始消融实验 (Baseline vs BicliqueGCN)...")
    %cd TSB_CL_Project
    
    print("\n" + "="*40)
    print("--- 1. 运行 Baseline (纯 LightGCN, 50 Epochs) ---")
    print("="*40)
    !python solo_model/train_1_lightgcn.py
    
    print("\n" + "="*40)
    print("--- 2. 运行 Biclique GCN (最大相似二团增强, tau=2, 50 Epochs) ---")
    print("="*40)
    !python solo_model/train_2_biclique_gcn.py